# Introduction to Machine Learning – Titanic Dataset

This notebook introduces basic supervised learning with:
- Preprocessing (missing values, encoding)
- Feature scaling
- Pipeline creation with Scikit-learn
- Model training & evaluation
- Model saving and serving with FastAPI

In [11]:
# 📦 Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

In [12]:
# 📥 Load Titanic Dataset
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
# 🧹 Select Features and Target
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
target = 'Survived'

X = df[features]
y = df[target]

In [14]:
# 🔧 Define Preprocessing Pipeline
numeric_features = ['Age', 'Fare']
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ['Pclass', 'Sex', 'Embarked']
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

In [15]:
# 🔁 Full Pipeline with Model
clf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
clf_pipeline.fit(X_train, y_train)

# Evaluate
y_pred = clf_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.84      0.82       105
           1       0.76      0.72      0.74        74

    accuracy                           0.79       179
   macro avg       0.78      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179



## Save the Trained Pipeline

In [16]:
joblib.dump(clf_pipeline, "titanic_pipeline.pkl")

['titanic_pipeline.pkl']

## Exercise 1: Try a Different Classifier
Replace the logistic regression model in the pipeline with another classifier, such as `RandomForestClassifier`, and compare the results.

```python
from sklearn.ensemble import RandomForestClassifier
# Replace the classifier in clf_pipeline
```

*What changes do you observe in precision and recall?*

In [18]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

# Remplir les valeurs manquantes
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Encoder les variables catégorielles
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])       # 'male' → 1, 'female' → 0
df['Embarked'] = LabelEncoder().fit_transform(df['Embarked'])

# Sélection des features
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


C:\Users\Admin\AppData\Local\Temp\ipykernel_21308\2835349595.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21308\2835349595.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Colonnes numériques uniquement
num_features = ['Age', 'Fare']
preprocessor = ColumnTransformer([
    ('scaler', StandardScaler(), num_features)
], remainder='passthrough')

# Pipeline avec RandomForest
clf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entraînement
clf_pipeline.fit(X_train, y_train)

# Prédictions
y_pred = clf_pipeline.predict(X_test)

# Rapport de classification
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.82      0.85      0.84       105
           1       0.77      0.74      0.76        74

    accuracy                           0.80       179
   macro avg       0.80      0.80      0.80       179
weighted avg       0.80      0.80      0.80       179



## Exercise 2: Use Cross-Validation
Apply cross-validation on the pipeline instead of a single train/test split.

```python
from sklearn.model_selection import cross_val_score
```

*Is the model stable across folds?*

In [21]:


# 🔹 Colonnes numériques à standardiser
num_features = ['Age', 'Fare']
preprocessor = ColumnTransformer([
    ('scaler', StandardScaler(), num_features)
], remainder='passthrough')  # les autres colonnes sont laissées telles quelles

# 🔹 Pipeline avec RandomForest
clf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# ✅ Validation croisée (5 folds)
scores = cross_val_score(clf_pipeline, X, y, cv=5, scoring='accuracy')

# 📊 Résultats
print("Scores pour chaque fold :", scores)
print("Moyenne des scores :", round(scores.mean(), 3))


Scores pour chaque fold : [0.78212291 0.80337079 0.83707865 0.79775281 0.83146067]
Moyenne des scores : 0.81


## Exercise 3: Add Feature Engineering
Add a new column to the Titanic data, such as `FamilySize = SibSp + Parch`, and evaluate if this feature improves the model.

```python
df['FamilySize'] = df['SibSp'] + df['Parch']
# Then include it in the feature list and re-run the pipeline
```

*Does the new feature improve the prediction metrics?*

In [22]:
# 1️⃣ Ajout de la variable FamilySize
df['FamilySize'] = df['SibSp'] + df['Parch']

# 2️⃣ Nouvelles features (on ajoute FamilySize)
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize']
X = df[features]
y = df['Survived']

# 3️⃣ Colonnes numériques à standardiser
num_features = ['Age', 'Fare', 'FamilySize']

# 4️⃣ Pipeline mis à jour
preprocessor = ColumnTransformer([
    ('scaler', StandardScaler(), num_features)
], remainder='passthrough')  # les autres colonnes sont passées telles quelles

clf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# 5️⃣ Cross-validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf_pipeline, X, y, cv=5, scoring='accuracy')

# 6️⃣ Résultat
print("Scores avec FamilySize :", scores)
print("Moyenne des scores :", round(scores.mean(), 3))


Scores avec FamilySize : [0.78212291 0.82022472 0.85393258 0.78089888 0.81460674]
Moyenne des scores : 0.81


## Exercise 4 (Bonus): Create a Streamlit Interface
Build a simple Streamlit UI to load the trained model and predict survival based on user input.

```python
# Example streamlit interface
import streamlit as st
import joblib
import pandas as pd

model = joblib.load("titanic_pipeline.pkl")
Pclass = st.selectbox("Pclass", [1, 2, 3])
Sex = st.selectbox("Sex", ["male", "female"])
Age = st.slider("Age", 0, 100, 25)
Fare = st.slider("Fare", 0.0, 500.0, 32.0)
Embarked = st.selectbox("Embarked", ["S", "C", "Q"])

if st.button("Predict"):
    X_new = pd.DataFrame([[Pclass, Sex, Age, Fare, Embarked]],
                         columns=["Pclass", "Sex", "Age", "Fare", "Embarked"])
    pred = model.predict(X_new)
    st.write("Prediction:", "Survived" if pred[0] == 1 else "Did not survive")
```

👉 *Try running your Streamlit app locally.*

In [23]:
import joblib

clf_pipeline.fit(X, y) 
joblib.dump(clf_pipeline, "titanic_pipeline.pkl")


['titanic_pipeline.pkl']

In [ ]:
pip install streamlit